##### Copyright 2025 Patrick Loeber, Google LLC

In [ ]:

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Workshop: Build with Gemini (Part 1)

<a target="_blank" href="https://colab.research.google.com/github/patrickloeber/workshop-build-with-gemini/blob/main/01-text-prompting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This workshop teaches how to build with Gemini using the Gemini API and Python SDK.

Course outline:

- **Part 1 (this notebook): Quickstart + Text prompting**
  - Text generation
  - Token counting
  - Streaming response
  - Chats
  - System prompts
  - Configuration parameters
  - Long context
  - Final excercise: Chat with book

- **[Part 2: Multimodal capabilities (image, video, audio, docs, code)](https://github.com/patrickloeber/workshop-build-with-gemini/blob/main/02-multimodal-capabilities.ipynb)**

- **[Part 3: Thinking models + agentic capabilities (tool usage)](https://github.com/patrickloeber/workshop-build-with-gemini/blob/main/03-thinking-and-tools.ipynb)**

## 0. Use the Google AI Studio as playground

Explore and play with all models in the [Google AI Studio](https://aistudio.google.com/apikey).


## 1. Setup


Install the [Google Gen AI Python SDK](https://github.com/googleapis/python-genai)

In [ ]:
%pip install -q -U google-genai

Get a free API key in the [Google AI Studio](https://aistudio.google.com/apikey).

Configure the API key, the client, and define a model:

In [ ]:
from google import genai
from google.genai import types
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
else:
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')


client = genai.Client(api_key=GEMINI_API_KEY)

# MODEL = "gemini-2.5-pro-preview-06-05"  # paid tier
# MODEL = "gemini-2.5-flash-preview-05-20"
# MODEL = "gemini-2.0-flash-lite"
MODEL = "gemini-2.0-flash"

 See all [models](https://ai.google.dev/gemini-api/docs/models).

## 2. Send your first prompt

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents="Create 3 names for a vegan restaurant"
)

print(response.text)

## 3. Token counting

Count tokens before generation:

In [ ]:
prompt = "The quick brown fox jumps over the lazy dog."

print(f"# charakters {len(prompt)}")
print(f"# words {len(prompt.split())}")
print(f"# tokens: ~{int(len(prompt.split()) * 4/3)}")   # rule of thumb: 100tokens=75words

# Count tokens in the input
token_count = client.models.count_tokens(
    model=MODEL, 
    contents=prompt
)
print(f"Input tokens: {token_count.total_tokens}")

# Estimate cost (example pricing for 2.0 Flash - check current rates)
estimated_cost = token_count.total_tokens * 0.10 / 1_000_000
print(f"Estimated input cost: ${estimated_cost:.6f}")

Count tokens after generation:

In [ ]:
prompt = "Write a haiku about artificial intelligence."

response = client.models.generate_content(
    model=MODEL,
    contents=prompt
)

print(response.text)

# Access token usage metadata
usage = response.usage_metadata
print(f"Input tokens: {usage.prompt_token_count}")
print(f"Thought tokens: {usage.thoughts_token_count}")
print(f"Output tokens: {usage.candidates_token_count}")
print(f"Total tokens: {usage.total_token_count}")

# Calculate total estimated cost
thought_tokens = usage.thoughts_token_count if usage.thoughts_token_count else 0
total_cost = (usage.prompt_token_count * 0.10 + (usage.candidates_token_count + thought_tokens) * 0.4) / 1_000_000
# total_cost = (usage.prompt_token_count * 0.15 + (usage.candidates_token_count + thought_tokens) * 3.5) / 1_000_000
print(f"Total estimated cost: ${total_cost:.6f}")

## 4. Text generation

The simplest way to generate text is to provide the model with a text-only prompt. `contents` can be a single prompt, a list of prompts, or a combination of multimodal inputs.

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    #contents="Create 3 names for a vegan restaurant",
    #contents=["Create 3 names for a vegan restaurant"],
    contents=["Create 3 names for a vegan restaurant", "city: Berlin"]
)

print(response.text)

#### Streaming response

By default, the model returns a response after completing the entire text generation process. You can achieve faster interactions by using streaming to return outputs as they're generated.

In [ ]:
response = client.models.generate_content_stream(
    model=MODEL,
    contents=["Explain how AI works"]
)

for chunk in response:
    print(chunk.text, end="")

#### Chat

The SDK chat class provides an interface to keep track of conversation history. Behind the scenes it uses the same `generate_content` method.

In [ ]:
chat = client.chats.create(model=MODEL)

response = chat.send_message("I have 2 dogs in my house.")
print(response.text)

In [ ]:
response = chat.send_message("I have 2 poodles")
print(response.text)

## 5. Configuration parameters

Every prompt you send to the model includes parameters that control how the model generates responses. You can configure these parameters, or let the model use the default options.

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=1024,
        temperature=1.0,
        top_p=0.95,  # Nucleus sampling - diversity of token selection
        top_k=40,    # Consider top 40 most likely tokens
        stop_sequences=None,
        seed=1234,
    )
)
print(response.text)

- `max_output_tokens`: Prevents overly long responses and controls costs
- `temperature`: [0, 2]. Controls randomness. Use <0.4 for factual content, >0.7 for creative content
- `top_p`: [0, 1]. Controls diversity. Lower values = more focused, higher = more diverse
- `top_k`: Limits token choices. Lower = more focused, higher = more diverse
- `stop_sequences`: List of strings (up to 5) that tells the model to stop generating text if one of the strings is encountered in the response.
- `seed`: If specified, the model makes a best effort to provide the same response for repeated requests.

#### System instructions

System instructions let you steer the behavior of a model based on your specific use case. When you provide system instructions, you give the model additional context to help it understand the task and generate more customized responses. The model should adhere to the system instructions over the full interaction with the user, enabling you to specify product-level behavior separate from the prompts provided by end users.

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    # config=types.GenerateContentConfig(system_instruction="You are a Dumbledore."),
    config={"system_instruction": "You are Dumbledore."},
    contents="Hello there"
)

print(response.text)

## 6. Long context

Gemini 2.0 and 2.5 models have a 1M token context window.

In practice, 1 million tokens could look like:

- 50,000 lines of code (with the standard 80 characters per line)
- All the text messages you have sent in the last 5 years
- 8 average length English novels
- 1 hour of video data

Let's feed in an entire book and ask questions:



In [ ]:
import requests
res = requests.get("https://gutenberg.org/cache/epub/16317/pg16317.txt")
book = res.text

In [ ]:
print(book[:100])

In [ ]:
print(f"# charakters {len(book)}")
print(f"# words {len(book.split())}")
print(f"# tokens: ~{int(len(book.split()) * 4/3)}")   # rule of thumb: 100tokens=75words

In [ ]:
prompt = f"""Summarize the book.

Book:
{book}
"""

response = client.models.generate_content(
    model=MODEL,
    contents=prompt
)

print(response.text)

## !! Exercise: Chat with a book !!

Create an interactive chat session where you can "talk" to the book "Alice in Wonderland". You'll set up the chat with a specific persona for the AI and use the book's text as context for the conversation.

Tasks: 
- Download the text of "Alice in Wonderland" (helper code block is provided).
- Create a chat session using `client.chats.create()`:
- Use a system prompt: `"You are an expert book reviewer with a witty tone."`
- Use a temperature of `1.2`
- Send an initial message to the chat session using `chat.send_message()`:
- Send at least one follow-up question to the chat session and print its response.

In [ ]:
res = requests.get("https://gutenberg.org/cache/epub/28885/pg28885.txt")
book = res.text
print(f"# tokens: ~{int(len(book.split()) * 4/3)}")

In [ ]:
# TODO: create a chat and ask questions about the book

## Recap & Next steps

Nice work! You learned:
- Python SDK quickstart
- Text prompting
- Token counting
- Streaming and chats
- System prompts and config options
- Long context

Key Takeaways:
- Monitor token usage to control costs and stay within limits
- Use streaming for interactive applications and long responses
- Configure parameters based on your use case (factual vs creative content)
- System instructions are powerful for setting behavior and tone

More helpful resources:
- [Text Generation Guide](https://ai.google.dev/gemini-api/docs/text-generation)
- [Token Counting Guide](https://ai.google.dev/gemini-api/docs/tokens)
- [Long Context Documentation](https://ai.google.dev/gemini-api/docs/long-context)

Next steps:
- [Part 2: Multimodal capabilities (image, video, audio, docs, code)](https://github.com/patrickloeber/workshop-build-with-gemini/blob/main/02-multimodal-capabilities.ipynb)